In [67]:
from arcgis.gis import GIS
gis = GIS("home")

In [68]:
import arcpy
import requests
import io
import os


## Part 1A - Convert .las to DEM and TIN

### Set file paths

In [70]:
#set file paths for Part 1 of lab

#probably need to reset this when switching from home desktop to laptop
#laptop
#gdb_folder = r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb"
#data_path = r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\GIS5571_Lab2"

#desktop:
data_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\GIS5571_Lab2"
gdb_folder = os.path.join(data_path, "GIS5571_Lab2.gdb")

las_path = os.path.join(data_path, "DNR_example.las")




### Create las file request and file

In [71]:
#set url for las download, request
las_url = "https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-13-04.las"
las_res = requests.get(las_url)

#write to file - no need to zip for small example file
with open(las_path, "wb") as f:
    f.write(las_res.content)

In [73]:
#convert las to TIN with las dataset to TIN tool - too large to do so without thinning

TIN_path = os.path.join(data_path, "DNR_example_TIN")

arcpy.ddd.LasDatasetToTin(
    in_las_dataset= las_path,
    out_tin= TIN_path,
    thinning_type="RANDOM",
    thinning_method="PERCENT",
    thinning_value=10,
    max_nodes=5000000,
    z_factor=1,
    clip_to_extent="CLIP"
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\DNR_example_TIN'>

In [74]:
#Convert las to DEM with las dataset to raster tool - DEM is just a raster showing elevation

dem_path = os.path.join(gdb_folder, "DNR_example_DEM")

arcpy.conversion.LasDatasetToRaster(
    in_las_dataset= las_path,
    out_raster= dem_path,
    value_field="ELEVATION",
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\GIS5571_Lab2.gdb\\DNR_example_DEM'>

### Control which layer is visible and export to pdf

In [75]:
#Set project and working map
project = arcpy.mp.ArcGISProject("CURRENT")
#desktop version
part1_map = project.listMaps("Map")[0]

#laptop version
#part1_map = project.listMaps("Map2")[0]

#prepare layers and layouts for selection
tin_layer = part1_map.listLayers("DNR_example_TIN")[0]
dem_layer = part1_map.listLayers("DNR_example_DEM")[0]
part1_layout = project.listLayouts("TIN")[0]

#Turn on TIN, turn off DEM, export layout
tin_layer.visible = True
dem_layer.visible = False
part1_layout.exportToPDF(os.path.join(data_path, "TIN_export.pdf"))

#Turn off TIN,turn on DEM, export layout
tin_layer.visible = False
dem_layer.visible = True
part1_layout.exportToPDF(os.path.join(data_path, "DEM_export.pdf"))

'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\GIS5571_Lab2\\DEM_export.pdf'